In [1]:
import requests
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import csv
import datetime
import re

betfair_SPFL = "https://www.betfair.com/sport/football/scottish-premiership/105"
fixtures_URL = "http://api.clubelo.com/Fixtures"
edge_threshold = 0.04


def fractional_to_decimal(fractional):
    fractional = fractional.strip("\n")
    
    if not re.match('(?:[1-9][0-9]*)(\.[0-9]*)?\/[1-9][0-9]*(\.[0-9]*)?', fractional):
        return np.NaN
    
    first, second = [float(x) for x in fractional.split("/") if x]
    
    return first/second+1.0

def current_odds_betfair(url):

    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    events = soup.findAll('div', attrs={"class":"event-information"})
    matches = []

    for event in events:
        teams = event.findAll("span", attrs={"class":"team-name"})
        home_name = teams[0].attrs['title']
        away_name = teams[1].attrs['title']
        home_name = home_name.replace("Utd", "United")
        away_name = away_name.replace("Utd", "United")
        home_name = home_name.replace("Co", "County")
        away_name = away_name.replace("Co", "County")
        
        odds = event.findAll("span", attrs={"class":"ui-runner-price"})

        if len(odds) == 5:
            over = fractional_to_decimal(odds[0].text)
            under = fractional_to_decimal(odds[1].text)
            home_odds = fractional_to_decimal(odds[2].text)
            draw_odds = fractional_to_decimal(odds[3].text)
            away_odds = fractional_to_decimal(odds[4].text)
        if len(odds) == 3:
            home_odds = fractional_to_decimal(odds[0].text)
            draw_odds = fractional_to_decimal(odds[1].text)
            away_odds = fractional_to_decimal(odds[2].text)

        matches.append((home_name, away_name, home_odds, draw_odds, away_odds))

    return pd.DataFrame(matches, columns=('Home', "Away", "O(W)", "O(D)", "O(L)"))

def current_prob_elo():
    
    # def current_prob():
    away = ["GD<-5","GD=-5","GD=-4","GD=-3","GD=-2","GD=-1"]
    draw = ["GD=0"]
    home = ["GD=1","GD=2","GD=3","GD=4","GD=5","GD>5"]
    over = ["GD<-5","GD=-5","GD=-4","GD=-3","GD=3","GD=4","GD=5","GD>5"]
    under = ["GD=-2","GD=-1","GD=0","GD=1","GD=2"]

    response = requests.get(fixtures_URL)
    decoded_content = response.content.decode('utf-8')
    fixtures = pd.DataFrame(csv.reader(decoded_content.splitlines(), delimiter=','))
    fixtures.columns = fixtures.iloc[0]
    fixtures = fixtures.drop(0)

    fixtures = fixtures[fixtures['Country'] == 'SCO']

    fixtures["P(W)"] = fixtures[home].astype(float).sum(axis=1)
    fixtures["P(D)"] = fixtures[draw].astype(float).sum(axis=1)
    fixtures["P(L)"] = fixtures[away].astype(float).sum(axis=1)
    fixtures["P(over2.5)"] = fixtures[over].astype(float).sum(axis=1)
    fixtures["P(under2.5)"] = fixtures[under].astype(float).sum(axis=1)

    fixtures = fixtures[["Date","Home", "Away", "P(W)", "P(D)", "P(L)", "P(over2.5)", "P(under2.5)"]]
    
    return fixtures

def current_prob(league='Scottish Premiership'):
    response = requests.get("https://projects.fivethirtyeight.com/soccer-api/club/spi_matches.csv")
    soup = BeautifulSoup(response.text, 'html.parser')

    decoded_content = response.content.decode('utf-8')
    fixtures = pd.DataFrame(csv.reader(decoded_content.splitlines(), delimiter=','), )
    fixtures.columns = fixtures.iloc[0]
    fixtures = fixtures.drop(0)

    if league:
        fixtures = fixtures[fixtures['league'] == league]
    fixtures['date'] = pd.to_datetime(fixtures['date'])
    fixtures = fixtures[fixtures['date'] > datetime.datetime.now()]


    fixtures["P(W)"] = fixtures['prob1'].astype(float)
    fixtures["P(D)"] = fixtures['probtie'].astype(float)
    fixtures["P(L)"] = fixtures['prob2'].astype(float)
    fixtures = fixtures.rename(columns={'date':'Date', 'team1':'Home', 'team2':'Away'})

    fixtures = fixtures[["Date","Home", "Away", "P(W)", "P(D)", "P(L)"]]
    return fixtures
    
def color_negative(v, color, limit):
        return f"color: {color};" if v < limit else "color: green;font-weight: bold"
    
def display_summary(bet_amount=1, url=betfair_SPFL, league='Scottish Premiership', filter=False):
    if filter:
        summary_df = filter_dataframe(summary(bet_amount, url,league).round(2))
    else:
        summary_df = summary(bet_amount, url, league).round(2)
        
    summary_df = summary_df.drop_duplicates()
    
    
    styler = summary_df.style.applymap(color_negative, color='red', limit=0, subset=['Payoff(W)', 'Payoff(D)', 'Payoff(L)'])
    styler = styler.applymap(color_negative, color='red', limit=edge_threshold, subset=['Edge(W)', 'Edge(D)', 'Edge(L)'])
    styler.format({
        'P(W)': '{:,.2%}'.format,
        'P(D)': '{:,.2%}'.format,
        'P(L)': '{:,.2%}'.format,
        'Edge(W)': '{:,.2%}'.format,
        'Edge(D)': '{:,.2%}'.format,
        'Edge(L)': '{:,.2%}'.format,
    })
    
    return styler

def overall_odds():
    URLS = ["https://www.betfair.com/sport/football/english-premier-league/10932509",
           "https://www.betfair.com/sport/football/italian-serie-a/81",
           "https://www.betfair.com/sport/football/french-ligue-1/55",
           "https://www.betfair.com/sport/football/spanish-la-liga/117",
           "https://www.betfair.com/sport/football/french-ligue-2/57",
           "https://www.betfair.com/sport/football/turkish-super-league/194215",
           "https://www.betfair.com/sport/football/spanish-segunda-division/12204313",
           "https://www.betfair.com/sport/football/portuguese-primeira-liga/99",
           "https://www.betfair.com/sport/football/greek-super-league/67",
           "https://www.betfair.com/sport/football/italian-serie-b/12199689",
           "https://www.betfair.com/sport/football/italian-serie-a/81",
           "https://www.betfair.com/sport/football/mexican-liga-mx/5627174"]
    
    return pd.concat([current_odds_betfair(url) for url in URLS])

def summary(bet_amount=1,url=betfair_SPFL,league='Scottish Premiership'):
    
    prob = current_prob(league=league)
    if url:
        odds = current_odds_betfair(url)
    else:
        odds = overall_odds()
        
    summary_df = odds.merge(prob)
    
    summary_df["Payoff(W)"] = (summary_df["P(W)"]*(summary_df["O(W)"]-bet_amount))-((1-summary_df["P(W)"])*bet_amount)
    summary_df["Payoff(D)"] = (summary_df["P(D)"]*(summary_df["O(D)"]-bet_amount))-((1-summary_df["P(D)"])*bet_amount)
    summary_df["Payoff(L)"] = (summary_df["P(L)"]*(summary_df["O(L)"]-bet_amount))-((1-summary_df["P(L)"])*bet_amount)
    
    summary_df["Edge(W)"] = summary_df["P(W)"] - (1/summary_df["O(W)"])
    summary_df["Edge(D)"] = summary_df["P(D)"] - (1/summary_df["O(D)"])
    summary_df["Edge(L)"] = summary_df["P(L)"] - (1/summary_df["O(L)"])

    return summary_df.head(len(summary_df))#.iloc[:,[5,0,1,6,8,11,7,9,12,8,10,13]]

def filter_dataframe(df):
    
    frames = [df[(df['Payoff(W)'] > 0) & (df['Edge(W)'] >= edge_threshold)],
              df[(df['Payoff(D)'] > 0) & (df['Edge(D)'] >= edge_threshold)],
              df[(df['Payoff(L)'] > 0) & (df['Edge(L)'] >= edge_threshold)]]
    
    return pd.concat(frames)


In [2]:
display_summary(bet_amount=1, url=None, league=None, filter=True)

,Home,Away,O(W),O(D),O(L),Date,P(W),P(D),P(L),Payoff(W),Payoff(D),Payoff(L),Edge(W),Edge(D),Edge(L)
17,Fiorentina,AC Milan,2.750000,3.250000,2.500000,2023-03-04 00:00:00,40.00%,26.00%,34.00%,0.100000,-0.160000,-0.150000,4.00%,-5.00%,-6.00%
30,Strasbourg,Brest,2.100000,3.100000,3.700000,2023-03-05 00:00:00,52.00%,25.00%,23.00%,0.090000,-0.230000,-0.130000,4.00%,-8.00%,-4.00%
36,Almeria,Villarreal,3.300000,3.600000,nan,2023-03-04 00:00:00,34.00%,26.00%,40.00%,0.120000,-0.050000,nan,4.00%,-1.00%,nan%
51,Benevento,Sudtirol,2.800000,3.000000,2.620000,2023-03-01 00:00:00,44.00%,35.00%,21.00%,0.230000,0.050000,-0.440000,8.00%,2.00%,-17.00%
34,Villarreal,Getafe,1.670000,3.500000,5.500000,2023-02-27 00:00:00,46.00%,30.00%,24.00%,-0.230000,0.050000,0.310000,-14.00%,1.00%,6.00%
41,Amiens,Bordeaux,2.880000,3.200000,2.450000,2023-02-27 00:00:00,27.00%,28.00%,44.00%,-0.210000,-0.090000,0.090000,-7.00%,-3.00%,4.00%


In [4]:
display_summary(bet_amount=1, url="https://www.betfair.com/sport/football/scottish-premiership/105", league="Scottish Premiership")

,Home,Away,O(W),O(D),O(L),Date,P(W),P(D),P(L),Payoff(W),Payoff(D),Payoff(L),Edge(W),Edge(D),Edge(L)
0,St Johnstone,St Mirren,2.880000,3.000000,2.550000,2023-02-25 00:00:00,42.00%,29.00%,29.00%,0.190000,-0.130000,-0.250000,7.00%,-4.00%,-10.00%
1,Aberdeen,Livingston,1.800000,3.500000,4.400000,2023-02-25 00:00:00,39.00%,26.00%,35.00%,-0.300000,-0.100000,0.560000,-17.00%,-3.00%,13.00%
2,Kilmarnock,Motherwell,2.550000,3.000000,2.900000,2023-02-25 00:00:00,37.00%,27.00%,36.00%,-0.060000,-0.190000,0.050000,-2.00%,-6.00%,2.00%
